# Preparation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd, numpy as np
from plotly import express as px
#  pip install -U kaleido

In [ ]:
df = pd.read_pickle("/content/drive/MyDrive/CMG - Crystal Prediction Project/Ternary Materials Point Group Prediction/Data/NOMAD_2/Classification_Data_8.pkl").dropna()
print(df.shape)
df.head()

(1381099, 24)


,Atomic Number 1,Atomic Number 2,Atomic Number 3,Element_1,Element_2,Element_3,Coefficient 1,Coefficient 2,Coefficient 3,formula_reduced,bravais_lattice,crystal_system,space_group_number,point_group,lattice_parameters,Oxidation 1,Oxidation 2,Oxidation 3,IonicRadius_1,IonicRadius_2,IonicRadius_3,IonizationPot1st_1,IonizationPot1st_2,IonizationPot1st_3
0,3,5,41,Li,B,Nb,4,1,1,BLi4Nb,cF,cubic,216,-43m,"{'a': 6.71860936e-10, 'b': 6.71860936e-10, 'c'...",1,-1,-3,0.755440,1.048943,1.842691,5.3917,8.298,6.7589
1,4,5,41,Be,B,Nb,4,1,1,BBe4Nb,cF,cubic,216,-43m,"{'a': 6.119642980000002e-10, 'b': 6.1196429800...",1,-1,-3,0.643658,1.048943,1.842691,9.3226,8.298,6.7589
2,4,5,41,Be,B,Nb,2,1,1,BBe2Nb,cF,cubic,216,-43m,"{'a': 5.458027e-10, 'b': 5.458027e-10, 'c': 5....",2,-1,-3,0.452080,1.048943,1.842691,9.3226,8.298,6.7589
3,4,5,41,Be,B,Nb,2,1,1,BBe2Nb,cF,cubic,225,m-3m,"{'a': 5.420156379999999e-10, 'b': 5.4201563799...",2,-1,-3,0.452080,1.048943,1.842691,9.3226,8.298,6.7589
4,4,5,41,Be,B,Nb,2,1,1,BBe2Nb,mS,monoclinic,12,2/m,"{'a': 4.660283620000524e-10, 'b': 8.41781306e-...",2,-1,-3,0.452080,1.048943,1.842691,9.3226,8.298,6.7589


**Note** Change `trigonal` to `hexagonal`

In [ ]:
df['crystal_system'].replace(to_replace="trigonal", value="hexagonal", inplace=True)

In [ ]:
FEATURE_NAMES = [
#     "Atomic Number 1","Atomic Number 2","Atomic Number 3",
    "Coefficient 1", "Coefficient 2", "Coefficient 3", 
    "IonizationPot1st_1", "IonizationPot1st_2", "IonizationPot1st_3", 
    "Oxidation 1", "Oxidation 2", "Oxidation 3", # Used to be electronegativity 
    "IonicRadius_1", "IonicRadius_2", "IonicRadius_3"
]

GROUPERS = [
    "Atomic Number 1","Atomic Number 2","Atomic Number 3",
    "Coefficient 1", "Coefficient 2", "Coefficient 3"
]

# Y_NAME = 'crystal_system'
# Y_NAME = 'bravais_lattice'
# Y_NAME = 'crystal_system'
Y_NAME = 'space_group_number'

# SG Filtered + Edge


Filter

In [ ]:
Y_NAME = 'space_group_number'
df[Y_NAME] = df[Y_NAME].astype(int).astype(str)
accepted_classes = (df[Y_NAME].value_counts() > 300).replace(False, np.nan).dropna().index.tolist()
df_filtered = df[df[Y_NAME].isin(accepted_classes)].copy()

print("Previous Data Size:", df.shape[0])
print("New Data Size:", df_filtered.shape[0])
print("Number of Classes Left:", len(accepted_classes))

Previous Data Size: 1381099
New Data Size: 1372271
Number of Classes Left: 50


In [ ]:
# print(set(df['bravais_lattice'].unique()) - set(df_filtered['bravais_lattice'].unique()))
# df['bravais_lattice'].value_counts()['cI']

In [ ]:
df_filtered['space_group_number'].unique().shape
df_filtered['bravais_lattice'].unique().shape

(13,)

In [ ]:
df_dedup = df_filtered[FEATURE_NAMES+[Y_NAME]].drop_duplicates().copy()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
# from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

df_grouped2 = df_dedup.groupby(FEATURE_NAMES)[Y_NAME].apply(list).reset_index()

In [ ]:
df_grouped2[Y_NAME]

0                  [216, 107, 156]
1          [216, 44, 42, 107, 156]
2         [216, 194, 42, 107, 156]
3              [216, 44, 107, 156]
4          [216, 42, 44, 107, 156]
                    ...           
320633                         [1]
320634                         [1]
320635                       [148]
320636                         [1]
320637                         [2]
Name: space_group_number, Length: 320638, dtype: object

In [ ]:
ml_binner = MultiLabelBinarizer()
ml_binner.fit(df_grouped2[Y_NAME])
y = ml_binner.transform(df_grouped2[Y_NAME])
X = df_grouped2[FEATURE_NAMES].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
clf = RandomForestClassifier(n_estimators=100, random_state=0, class_weight='balanced_subsample')
clf.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced_subsample', random_state=0)

In [ ]:
y_pred = clf.predict(X_test)


In [ ]:
y_test_inv = ml_binner.inverse_transform(y_test)
y_pred_inv = ml_binner.inverse_transform(y_pred)

In [ ]:
y_test_filtered = []
y_pred_filtered = []

for test_labels, pred_labels in zip(y_test_inv, y_pred_inv):
    test_sample_filt = [elem for elem in test_labels if elem.isdigit()]
    pred_sample_filt = [elem for elem in pred_labels if elem.isdigit()]
    
    y_test_filtered.append(test_sample_filt)
    y_pred_filtered.append(pred_sample_filt)

In [ ]:
ml_binner = MultiLabelBinarizer()
ml_binner.fit(y_test_filtered)

y_test_filtered_bins = ml_binner.transform(y_test_filtered)
y_pred_filtered_bins = ml_binner.transform(y_pred_filtered)

In [ ]:
from sklearn.metrics import classification_report
target_names =  ml_binner.classes_
print(classification_report(y_test_filtered_bins, y_pred_filtered_bins, target_names=[str(i) for i in target_names]))

In [ ]:
from sklearn.metrics import classification_report
target_names =  ml_binner.classes_# [elem for elem in ml_binner.classes_ if elem.isdigit()]
print(classification_report(y_test, y_pred, target_names=[str(i) for i in target_names]))

              precision    recall  f1-score   support

           1       0.57      0.13      0.22       223
          10       0.91      0.93      0.92     12553
         107       0.98      0.99      0.98      4688
          11       0.42      0.10      0.16       114
         119       0.93      0.90      0.92     12325
          12       0.88      0.90      0.89     12619
         123       0.97      0.95      0.96     13785
         127       0.79      0.37      0.51       102
         129       0.28      0.30      0.29       162
         139       0.85      0.89      0.87     11762
          14       0.54      0.21      0.31       427
         140       0.92      0.15      0.26        79
         141       0.54      0.27      0.36       110
         148       0.84      0.50      0.63       274
          15       0.72      0.23      0.35       290
         156       0.94      0.97      0.96      4550
         160       0.37      0.07      0.12      3823
         164       0.66    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.

